In [317]:
import pandas as pd
import ast
import re
df_movies = pd.read_parquet('datasets/clean_movies_dataset.parquet')
df_credits = pd.read_parquet('datasets/credits.parquet')

In [318]:
df_credits.cast.apply(lambda x : type(x))

0        <class 'str'>
1        <class 'str'>
2        <class 'str'>
3        <class 'str'>
4        <class 'str'>
             ...      
45471    <class 'str'>
45472    <class 'str'>
45473    <class 'str'>
45474    <class 'str'>
45475    <class 'str'>
Name: cast, Length: 45476, dtype: object

In [319]:
df_movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return'],
      dtype='object')

### def cantidad_filmaciones_mes( Mes ): Se ingresa un mes en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en el mes consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en el mes de X

In [320]:
def cantidad_filmaciones_mes(mes: str):
    mes = mes.lower()
    df_movies.release_date = pd.to_datetime(df_movies.release_date)
    def mes_a_numero (mes):
        meses = {
            'enero':1,'febrero':2,'marzo':3,'abril':4,
            'mayo':5,'junio':6,'julio':7,'agosto':8,
            'septiembre':9,'octubre':10,'noviembre':11,'diciembre':12
            }
        return meses[mes]
    cantidad = df_movies.release_date.loc[df_movies.release_date.dt.month == mes_a_numero(mes)].count()
    return f'{cantidad.item()} cantidad de películas fueron estrenadas en el mes de {mes}'

- prueba

In [321]:
cantidad_filmaciones_mes('febrero')

'3032 cantidad de películas fueron estrenadas en el mes de febrero'

### def cantidad_filmaciones_dia( Dia ): Se ingresa un día en idioma Español. Debe devolver la cantidad de películas que fueron estrenadas en día consultado en la totalidad del dataset.
                    Ejemplo de retorno: X cantidad de películas fueron estrenadas en los días X

In [322]:
def cantidad_filmaciones_dia(dia : str):
    dia = dia.lower()
    df_movies.release_date = pd.to_datetime(df_movies.release_date, errors= 'coerce')
    def dia_numero(dia_numero):
        dias = {'lunes' : 0,'martes' : 1, 'miercoles': 2, 'jueves':3, 'viernes':4, 'sabado':5, 'domingo':6}
        return dias[dia_numero]
    cantidad = df_movies.release_date.loc[df_movies.release_date.dt.weekday == dia_numero(dia)].count()
    return {'dia' : dia , 'cantidad' : cantidad.item()}

- prueba

In [323]:
cantidad_filmaciones_dia('lunes')

{'dia': 'lunes', 'cantidad': 3503}

### def score_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, el año de estreno y el score.
                    Ejemplo de retorno: La película X fue estrenada en el año X con un score/popularidad de

In [324]:
def score_titulo(titulo_de_la_filmacion: str):
    titulo_de_la_filmacion = titulo_de_la_filmacion.lower()
    respuesta = df_movies[['title','release_year','popularity']].loc[df_movies.title.apply(lambda x : x.lower()) == titulo_de_la_filmacion]
    titulo = respuesta['title'].values[0]
    anio = respuesta['release_year'].values[0]
    popularidad = respuesta['popularity'].values[0]
    return f"la pelicula {str(titulo)} fue estrenada en el año {str(anio)} con un score/popularidad de {str(popularidad)}"

- prueba

In [325]:
score_titulo('Toy Story')

'la pelicula Toy Story fue estrenada en el año 1995 con un score/popularidad de 21.946943'

### def votos_titulo( titulo_de_la_filmación ): Se ingresa el título de una filmación esperando como respuesta el título, la cantidad de votos y el valor promedio de las votaciones. La misma variable deberá de contar con al menos 2000 valoraciones, caso contrario, debemos contar con un mensaje avisando que no cumple esta condición y que por ende, no se devuelve ningun valor.
                    Ejemplo de retorno: La película X fue estrenada en el año X. La misma cuenta con un total de X valoraciones, con un promedio de X

In [326]:
def votos_titulo(titulo_de_la_filmacion: str):
    titulo_de_la_filmacion = titulo_de_la_filmacion.lower()
    respuesta = df_movies[['title','release_year','vote_count','vote_average']].loc[df_movies.title.apply(lambda x : x.lower()) == titulo_de_la_filmacion]
    titulo = respuesta['title'].values[0]
    anio = respuesta['release_year'].values[0]
    cantidad_votos = respuesta['vote_count'].values[0]
    promedio_votos = respuesta['vote_average'].values[0]
    if respuesta['vote_count'].values[0] <2000:
        return f'La variable ingresada no cumple con la condición necesaria de tener más de 2000 valoraciones'
    else:
        return f"la pelicula {str(titulo)} fue estrenada en el año {str(anio)}. La misma cuenta con {str(cantidad_votos)} votos con un promedio de {str(promedio_votos)}"


- prueba

In [327]:
votos_titulo('Toy Story')

'la pelicula Toy Story fue estrenada en el año 1995. La misma cuenta con 5415 votos con un promedio de 7.7'

### def get_actor( nombre_actor ): Se ingresa el nombre de un actor que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, la cantidad de películas que en las que ha participado y el promedio de retorno. La definición no deberá considerar directores.
                    Ejemplo de retorno: El actor X ha participado de X cantidad de filmaciones, el mismo ha conseguido un retorno de X con un promedio de X por filmación

In [328]:
def get_actor(nombre_actor: str):
    resultado = df_credits[df_credits['cast'].apply(lambda x : nombre_actor.lower().rstrip() in x.lower())]
    cantidad_proyectos = resultado.shape[0]
    lista_proyectos = resultado.id.tolist()
    proyectos = df_movies[df_movies['id'].isin(lista_proyectos)]
    retorno = sum(proyectos.revenue.tolist())
    promedio = round(retorno/cantidad_proyectos)
    return f'El actor {nombre_actor} ha participado de {cantidad_proyectos} filmaciones, el mismo ha conseguido un retorno de {retorno} con un promedio de {promedio} por filmacion'

- prueba

In [329]:
get_actor('Robin Williams')

'El actor Robin Williams ha participado de 83 filmaciones, el mismo ha conseguido un retorno de 6356609847 con un promedio de 76585661 por filmacion'

### def get_director( nombre_director ): Se ingresa el nombre de un director que se encuentre dentro de un dataset debiendo devolver el éxito del mismo medido a través del retorno. Además, deberá devolver el nombre de cada película con la fecha de lanzamiento, retorno individual, costo y ganancia de la misma.

In [344]:
df_movies.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count', 'release_year', 'return'],
      dtype='object')

In [363]:
def get_director(nombre_director: str):
    condicion1 = df_credits['crew'].str.contains(r'\b{}\b'.format(nombre_director.title()), regex=True)
    condicion2 = df_credits['crew'].str.contains(r'\b{}\b'.format('Director'), regex=True)
    resultado = df_credits[condicion1 & condicion2]
    lista_proyectos = resultado.id.tolist()
    proyectos = df_movies[df_movies['id'].isin(lista_proyectos)]
    retorno = sum(proyectos.revenue.tolist())
    peliculas = ""
    print(proyectos)
        #peliculas = f"{peliculas}{proyectos['title'].values[0]} - {proyectos['release_date'].values[0]} /n"
        
    return f'El director {nombre_director} ha conseguido un retorno de {retorno}, sus trabajos son: /n {peliculas}'
    

- prueba

In [364]:
get_director('Joe Johnston')

                                   belongs_to_collection     budget   
1                                                   None   65000000  \
552                                                 None   27000000   
1945   {'id': 72119, 'name': 'Honey, I Shrunk the Kid...   32000000   
1985                                                None   42000000   
2084                                                None   35000000   
2388                                                None          0   
4509   {'id': 328, 'name': 'Jurassic Park Collection'...   93000000   
7171                                                None  100000000   
7640                                                None          0   
8848   {'id': 133830, 'name': 'Star Wars: The Ewok Ad...    3000000   
14823                                               None  150000000   
17444  {'id': 131295, 'name': 'Captain America Collec...  140000000   
28062                                               None          0   

     

'El director Joe Johnston ha conseguido un retorno de 1531004802, sus trabajos son: /n '